In [22]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [2]:
# Load the dataset
data = pd.read_csv("kepler_data.csv")
data = data.drop(columns=['kepoi_name','kepler_name','koi_tce_delivname'])
data.fillna(0, inplace=True)

In [3]:
# Encode the target variable using label encoding
label_encoder = LabelEncoder()
data['koi_disposition'] = label_encoder.fit_transform(data['koi_disposition'])
data['koi_pdisposition'] = label_encoder.fit_transform(data['koi_pdisposition'])

In [4]:
data.head()

,kepid,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,1,0,1.000,0,0,0,0,9.488036,2.780000e-05,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,1,0,0.969,0,0,0,0,54.418383,2.480000e-04,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,0,0,0.000,0,0,0,0,19.899140,1.490000e-05,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,2,1,0.000,0,1,0,0,1.736952,2.630000e-07,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,1,0,1.000,0,0,0,0,2.525592,3.760000e-06,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [5]:
data.describe()

,kepid,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,9.564000e+03,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,...,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000
mean,7.690628e+06,1.258678,0.506796,0.404914,0.208595,0.232748,0.197512,0.120033,75.671358,0.002046,...,-154.070368,4.146565,0.114830,-0.136155,1.663099,0.344564,-0.375487,292.060163,43.810433,14.263114
std,2.653459e+06,0.828618,0.499980,0.471473,4.767290,0.422605,0.398142,0.325018,1334.744046,0.008058,...,79.293359,0.926530,0.132138,0.088897,6.018842,0.911166,2.116208,4.766657,3.601243,1.393033
min,7.574500e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.241843,0.000000,...,-1762.000000,0.000000,0.000000,-1.207000,0.000000,0.000000,-116.137000,279.852720,36.577381,0.000000
25%,5.556034e+06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.733684,0.000004,...,-195.000000,4.169000,0.038000,-0.195000,0.811000,0.109000,-0.232000,288.660770,40.777173,13.439750
50%,7.906892e+06,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.752831,0.000029,...,-157.000000,4.432000,0.066000,-0.123000,0.988000,0.239000,-0.105000,292.261125,43.677504,14.520000
75%,9.873066e+06,2.000000,1.000000,0.995000,0.000000,0.000000,0.000000,0.000000,40.715178,0.000245,...,-103.000000,4.539000,0.143000,-0.075000,1.313000,0.355000,-0.065000,295.859160,46.714611,15.322000
max,1.293514e+07,2.000000,1.000000,1.000000,465.000000,1.000000,1.000000,1.000000,129995.778400,0.173000,...,0.000000,5.364000,1.472000,0.000000,229.908000,33.091000,0.000000,301.720760,52.336010,20.003000


In [6]:
# Extract features (X) and target variable (y)
X = data.drop(columns=['koi_disposition'])  # Features
y = data['koi_disposition']  # Target variable

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   kepid              9564 non-null   int64  
 1   koi_disposition    9564 non-null   int64  
 2   koi_pdisposition   9564 non-null   int64  
 3   koi_score          9564 non-null   float64
 4   koi_fpflag_nt      9564 non-null   int64  
 5   koi_fpflag_ss      9564 non-null   int64  
 6   koi_fpflag_co      9564 non-null   int64  
 7   koi_fpflag_ec      9564 non-null   int64  
 8   koi_period         9564 non-null   float64
 9   koi_period_err1    9564 non-null   float64
 10  koi_period_err2    9564 non-null   float64
 11  koi_time0bk        9564 non-null   float64
 12  koi_time0bk_err1   9564 non-null   float64
 13  koi_time0bk_err2   9564 non-null   float64
 14  koi_impact         9564 non-null   float64
 15  koi_impact_err1    9564 non-null   float64
 16  koi_impact_err2    9564 

In [8]:
# Define features and target
X = data.drop('koi_disposition', axis=1)
y = data['koi_disposition']


In [9]:
y_train = y

In [10]:
data.describe()

,kepid,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,9.564000e+03,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,...,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000,9564.000000
mean,7.690628e+06,1.258678,0.506796,0.404914,0.208595,0.232748,0.197512,0.120033,75.671358,0.002046,...,-154.070368,4.146565,0.114830,-0.136155,1.663099,0.344564,-0.375487,292.060163,43.810433,14.263114
std,2.653459e+06,0.828618,0.499980,0.471473,4.767290,0.422605,0.398142,0.325018,1334.744046,0.008058,...,79.293359,0.926530,0.132138,0.088897,6.018842,0.911166,2.116208,4.766657,3.601243,1.393033
min,7.574500e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.241843,0.000000,...,-1762.000000,0.000000,0.000000,-1.207000,0.000000,0.000000,-116.137000,279.852720,36.577381,0.000000
25%,5.556034e+06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.733684,0.000004,...,-195.000000,4.169000,0.038000,-0.195000,0.811000,0.109000,-0.232000,288.660770,40.777173,13.439750
50%,7.906892e+06,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.752831,0.000029,...,-157.000000,4.432000,0.066000,-0.123000,0.988000,0.239000,-0.105000,292.261125,43.677504,14.520000
75%,9.873066e+06,2.000000,1.000000,0.995000,0.000000,0.000000,0.000000,0.000000,40.715178,0.000245,...,-103.000000,4.539000,0.143000,-0.075000,1.313000,0.355000,-0.065000,295.859160,46.714611,15.322000
max,1.293514e+07,2.000000,1.000000,1.000000,465.000000,1.000000,1.000000,1.000000,129995.778400,0.173000,...,0.000000,5.364000,1.472000,0.000000,229.908000,33.091000,0.000000,301.720760,52.336010,20.003000


In [11]:
# Split the data into training and testing sets with a fixed train size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42)

# RandomForestClassifier

In [12]:
# Initialize a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
# Train the classifier
clf.fit(X_train, y_train)
# Predict the labels for the test set
y_pred = clf.predict(X_test)
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[392  92   0]
 [ 68 420   2]
 [  0   0 939]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.81      0.83       484
           1       0.82      0.86      0.84       490
           2       1.00      1.00      1.00       939

    accuracy                           0.92      1913
   macro avg       0.89      0.89      0.89      1913
weighted avg       0.92      0.92      0.92      1913



# LightGBM classifier

In [13]:
# Initialize LightGBM classifier
clf = lgb.LGBMClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000965 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9310
[LightGBM] [Info] Number of data points in the train set: 7651, number of used features: 43
[LightGBM] [Info] Start training from score -1.402501
[LightGBM] [Info] Start training from score -1.409968
[LightGBM] [Info] Start training from score -0.673603
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Confusion Matrix:
[[394  90   0]
 [ 70 419   1]
 [  0   0 939]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.81      0.83       484
           1       0.82      0.86      0.84       490
           2       1.00      1.00      1.00       939

    accuracy                           0.92      1913
   macro avg       0.89      0.89      0.89      1913
weighted avg       0.92      0.92      0.92      1913



In [14]:
# Initialize Gradient Boosting classifier
clf = GradientBoostingClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[398  86   0]
 [ 66 423   1]
 [  0   2 937]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       484
           1       0.83      0.86      0.85       490
           2       1.00      1.00      1.00       939

    accuracy                           0.92      1913
   macro avg       0.89      0.89      0.89      1913
weighted avg       0.92      0.92      0.92      1913



In [15]:
# Initialize Support Vector Machines classifier
clf = SVC()

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[  0   0 484]
 [  0   0 490]
 [  0   0 939]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       484
           1       0.00      0.00      0.00       490
           2       0.49      1.00      0.66       939

    accuracy                           0.49      1913
   macro avg       0.16      0.33      0.22      1913
weighted avg       0.24      0.49      0.32      1913



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [16]:
# Build the neural network model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # Assuming 3 classes for output
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = [0 if pred[0] >= 0.5 else 1 if pred[1] >= 0.5 else 2 for pred in y_pred]
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - accuracy: 0.3914 - loss: 158945.2812 - val_accuracy: 0.5287 - val_loss: 13912.3477
Epoch 2/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - accuracy: 0.4344 - loss: 11433.3486 - val_accuracy: 0.5287 - val_loss: 29982.3906
Epoch 3/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step - accuracy: 0.4347 - loss: 17387.2129 - val_accuracy: 0.3734 - val_loss: 4521.6255
Epoch 4/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 435us/step - accuracy: 0.4424 - loss: 12077.9971 - val_accuracy: 0.3407 - val_loss: 21259.8066
Epoch 5/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.4309 - loss: 15663.7285 - val_accuracy: 0.5287 - val_loss: 15213.5039
Epoch 6/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.4602 - loss: 9329.1172 - val_accuracy: 0.3538 - val_loss: 9225.8018
Epoch 7/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step - accuracy: 0.4456 - loss: 9004.3076 - val_accuracy: 0.4308 - val_loss: 1971.7487
Epoch 8/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/

In [17]:
# Initialize K-Nearest Neighbors classifier
clf = KNeighborsClassifier()

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[204 179 101]
 [177 229  84]
 [206 182 551]]

Classification Report:
              precision    recall  f1-score   support

           0       0.35      0.42      0.38       484
           1       0.39      0.47      0.42       490
           2       0.75      0.59      0.66       939

    accuracy                           0.51      1913
   macro avg       0.49      0.49      0.49      1913
weighted avg       0.55      0.51      0.53      1913



In [18]:
# Define the RNN model
model = Sequential([
    Embedding(input_dim=X_train.shape[1], output_dim=64),
    SimpleRNN(64),
    Dense(3, activation='softmax')  # Assuming 3 classes for output
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [19]:
# Train the model
model.fit(X_train, y_train, epochs=35, batch_size=40, validation_split=0.1)

Epoch 1/35


173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6922 - loss: 0.5524 - val_accuracy: 0.7454 - val_loss: 0.3735
Epoch 2/35
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7485 - loss: 0.3731 - val_accuracy: 0.7689 - val_loss: 0.3692
Epoch 3/35
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7592 - loss: 0.3656 - val_accuracy: 0.7689 - val_loss: 0.3759
Epoch 4/35
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7804 - loss: 0.3976 - val_accuracy: 0.7598 - val_loss: 0.3698
Epoch 5/35
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7621 - loss: 0.3581 - val_accuracy: 0.7663 - val_loss: 0.3683
Epoch 6/35
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7592 - loss: 0.3649 - val_accuracy: 0.7467 - val_loss: 0.3736
Epoch 7/35
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7636 - loss: 0.3573 - val_accuracy: 0.7663 - val_loss: 0.3709
Epoch 8/35
173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7665 - loss: 0.3662 - val_accuracy: 0.7702 - val_

In [20]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = [0 if pred[0] >= 0.5 else 1 if pred[1] >= 0.5 else 2 for pred in y_pred]
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix:
[[386  98   0]
 [ 97 388   5]
 [  4   0 935]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.80      0.80       484
           1       0.80      0.79      0.80       490
           2       0.99      1.00      1.00       939

    accuracy                           0.89      1913
   macro avg       0.86      0.86      0.86      1913
weighted avg       0.89      0.89      0.89      1913

